In [1]:
import rospy
import numpy as np
from sensor_msgs import msg
import cv2
import cv_bridge
from copy import deepcopy
import matplotlib.pyplot as plt
#import ipywidgets as widgets
from IPython.display import clear_output, Image, display
import PIL.Image
from cStringIO import StringIO
import dvrk
import PyKDL
import tf
from tf_conversions import posemath
import time
import math
import image_geometry
import threading


rospy.init_node('notebook')

bridge = cv_bridge.CvBridge()

left_image = None
left_image_msg = None
left_camera_info = None

right_image = None
right_image_msg = None
right_camera_info = None


global psm1, psm2, ecm

psm1 = None
psm2 = None
ecm = None

psm1 = dvrk.psm('PSM1')
psm2 = dvrk.psm('PSM2')
ecm = dvrk.ecm('ECM')

In [2]:
def left_image_callback(im_msg):
    global left_image, left_image_msg
    left_image = bridge.imgmsg_to_cv2(im_msg, desired_encoding='rgb8')
    left_image_msg = im_msg

def right_image_callback(im_msg):
    global right_image, right_image_msg
    right_image = bridge.imgmsg_to_cv2(im_msg, desired_encoding='rgb8')
    right_image_msg = bridge.cv2_to_imgmsg(right_image, encoding ='rgb8')

def left_camera_info_callback(camera_info_msg):
    global left_camera_info
    left_camera_info = camera_info_msg

def right_camera_info_callback(camera_info_msg):
    global right_camera_info
    right_camera_info = camera_info_msg

def tf_to_pykdl_frame(tfl_frame): #frane to translation and rotation
    pos, rot_quat = tfl_frame
    pos2 = PyKDL.Vector(*pos)
    rot = PyKDL.Rotation.Quaternion(*rot_quat)
    return PyKDL.Frame(rot, pos2)


def update_transforms(): #update the transforms
    global tf_cam_to_world, tf_world_to_ecm, tf_cam_to_ecm, tf_ecm_to_cam, tf_world_to_psm1, tf_world_to_psm2,tf_world_to_cam, tf_ecm_to_world, tf_world_to_ecm 
    tf_cam_to_world = tf_to_pykdl_frame(tf_listener.lookupTransform('simworld', 'Vision_sensor_left', rospy.Time()))
    tf_world_to_ecm = tf_to_pykdl_frame(tf_listener.lookupTransform('J1_ECM','simworld', rospy.Time()))
    tf_cam_to_ecm = tf_to_pykdl_frame(tf_listener.lookupTransform('J1_ECM', 'Vision_sensor_left',rospy.Time()))
    tf_ecm_to_cam = tf_to_pykdl_frame(tf_listener.lookupTransform('Vision_sensor_left', 'J1_ECM',rospy.Time()))
    tf_world_to_psm1 = tf_to_pykdl_frame(tf_listener.lookupTransform('J1_PSM1', 'simworld', rospy.Time()))
    tf_world_to_psm2 = tf_to_pykdl_frame(tf_listener.lookupTransform('J1_PSM2', 'simworld', rospy.Time()))
    tf_world_to_cam = tf_to_pykdl_frame(tf_listener.lookupTransform('Vision_sensor_left', 'simworld', rospy.Time()))
    tf_ecm_to_world = tf_to_pykdl_frame(tf_listener.lookupTransform('simworld', 'J1_ECM', rospy.Time()))
    tf_world_to_ecm = tf_to_pykdl_frame(tf_listener.lookupTransform('J1_ECM', 'simworld',rospy.Time()))

def get_contour(image, upper, lower):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    _, contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 2)
    #cv2.imshow(position,image)
    #cv2.waitKey(0)
    #plt.imshow(image)
    return contours       

def get_position_frame(contours, thresh_size): #find the position of the objects in the frame from a contour and sort them
    balls = []
    bowls = []
    for contour in contours:
        area = cv2.contourArea(contour)
        print(area)
        #bowl
        if area > thresh_size:
            M = cv2.moments(contour)
            bowl_lcX = int(M["m10"] / M["m00"])
            bowl_lcY = int(M["m01"] / M["m00"])
            bowl = [bowl_lcX, bowl_lcY]
            bowls.append(bowl)
            # draw the contour and center of the shape on the image
            #cv2.circle(contours, (bowl_lcX, bowl_lcY), 7, (255, 255, 255), -1)
            #plt.imshow(contours)
            #cv2.waitKey(0)
        if area < thresh_size and area > 1000:
            M = cv2.moments(contour)
            ball_lcX = int(M["m10"] / M["m00"])
            ball_lcY = int(M["m01"] / M["m00"])
            ball = [ball_lcX, ball_lcY]
            balls.append(ball)
            # draw the contour and center of the shape on the image
            #cv2.circle(contours, (ball_lcX, ball_lcY), 7, (255, 255, 255), -1)
            #plt.imshow(contours)
            #cv2.waitKey(0)
    
    balls.sort(key=lambda x: x[0])
    bowls.sort(key=lambda x: x[0])
    print('balls')
    print(balls)
    print('bowls')
    print(bowls)
    return balls, bowls

class Item:
    def __init__(self, world_coordinates):
        #self.color = color
        #self.right_frame_coordinates = [rx_coordinate, ry_coordinate]
        #self.left_frame_coordinates = [lx_coordinate, ly_coordinate]
        self.cam_coordinates = None
        self.ecm_coordinates = None
        self.world_coordinates = world_coordinates
        self.psm1_coordinates = None
        self.psm2_coordinates = None
        self.jaw1_coordinates = None
        self.jaw2_coordinates = None
        self.psm1_distance = None
        self.psm2_distance = None
        self.bowl1_distance = None
        self.bowl2_distance = None
        
    def get_world(self):
        return self.world_coordinates
    
def camera_to_world(right_coordinates, left_coordinates):  # 2D coordinates to 3D coordinates
    
    CV_TO_CAM_FRAME_ROT = np.asarray([
    [-1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
    ]) #transformation to fix frame orientation
    
    items = []
    i = 0
    for point in left_coordinates:
        disparity = abs(right_coordinates[i][0]-left_coordinates[i][0])
        #print(disparity)
        pos_cv = stereo_cam.projectPixelTo3d(left_coordinates[i], float(disparity))
        pos_cam = np.matmul(CV_TO_CAM_FRAME_ROT, pos_cv)
        update_transforms()
        pos_world = tf_cam_to_world * PyKDL.Vector(*pos_cam)
        items.append( Item(pos_world) )
        print(items[i].get_world())
        i+=1
        

    return items

def images_to_object_positions(left_image, right_image): #two images to two lists on ball and bowl objects
    global lower_red, upper_red
    
    lower_red = np.array([0,70,50])
    upper_red = np.array([180,255,255])
    
    left_contour = get_contour(left_image, upper_red, lower_red)
    right_contour = get_contour(right_image, upper_red, lower_red)
    
    left_balls, left_bowls = get_position_frame(left_contour, 2500)
    right_balls, right_bowls = get_position_frame(right_contour, 2500)
    
    if not len(left_balls) == len(right_balls) or not len(left_bowls) == len(right_bowls):
        raise e

    balls = camera_to_world(right_balls, left_balls)
    bowls = camera_to_world(right_bowls, left_bowls)
    
    return balls, bowls



def world_position_to_joints(items): # get the coordinates of an item wrt to relavant coordinate systems
    update_transforms()
    
    global PSM_J1_TO_BASE_LINK_TF
    
    
    PSM_J1_TO_BASE_LINK_ROT = PyKDL.Rotation.RPY(math.pi/2.0, -math.pi, 0)
    PSM_J1_TO_BASE_LINK_TF = PyKDL.Frame(PSM_J1_TO_BASE_LINK_ROT, PyKDL.Vector())

    ECM_J1_TO_BASE_LINK_ROT = PyKDL.Rotation.RPY(math.pi/2.0, -math.pi/2, 0)
    ECM_J1_TO_BASE_LINK_TF = PyKDL.Frame(ECM_J1_TO_BASE_LINK_ROT, PyKDL.Vector())
        
    for item in items:
        item.cam_coordinates = tf_world_to_cam * item.world_coordinates
        item.ecm_coordinates = PyKDL.Frame(PyKDL.Rotation.RPY(math.pi/2.0, -math.pi/2, 0), PyKDL.Vector()) * tf_world_to_ecm * item.world_coordinates
        item.psm1_coordinates = PyKDL.Frame(PyKDL.Rotation.RPY(math.pi/2.0, -math.pi, 0), PyKDL.Vector()) * tf_world_to_psm1 * item.world_coordinates
        item.psm2_coordinates = PyKDL.Frame(PyKDL.Rotation.RPY(math.pi/2.0, -math.pi, 0), PyKDL.Vector()) * tf_world_to_psm2 * item.world_coordinates
        #item.jaw1_coordinates = tf_world_to_jaw1* item.world_coordinates
        #item.jaw2_coordinates = tf_world_to_jaw2* item.world_coordinates

In [3]:
def distance(a,b):
    ad = math.sqrt(((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2))
    return ad

def pick_and_place(psm, ball, bowl): #one psm one bowl one ball
    global psm1, psm2

    approach_vec = PyKDL.Vector()


    if psm == psm1:
        ball_pos = ball.psm1_coordinates
        bowl_pos = bowl.psm1_coordinates
        pos1 = ball_pos + PyKDL.Vector(0,0,0.01) + approach_vec
        pos2 = ball_pos + approach_vec
        pos3 = ball_pos + PyKDL.Vector(0,0,0.05) + approach_vec
        pos4 = bowl_pos + PyKDL.Vector(0,0,0.05)
        
        psm1.open_jaw()
        psm1.move(pos1)
        psm1.move(pos2)
        psm1.close_jaw()
        psm1.move(pos3)
        psm1.move(pos4)
        psm1.open_jaw()
        
    elif psm == psm2:
        ball_pos = ball.psm2_coordinates
        bowl_pos = bowl.psm2_coordinates
        pos1 = ball_pos + PyKDL.Vector(0,0,0.01) + approach_vec
        pos2 = ball_pos + approach_vec
        pos3 = ball_pos + PyKDL.Vector(0,0,0.01) + approach_vec
        pos4 = bowl_pos + PyKDL.Vector(0,0,0.05)
        
        psm2.open_jaw()
        psm2.move(pos1)
        psm2.move(pos2)
        psm2.close_jaw()
        psm2.move(pos3)
        psm2.move(pos4)
        psm2.open_jaw()

def center_frame(pos_world, buff, delta):
    #global ecm, tf_cam_to_world
    xcentered = False
    ycentered = False
    while not xcentered & ycentered:
        ecm.move_joint_one(0,3)
        update_transforms()
        pos_camera = tf_world_to_cam * pos_world
        if  pos_camera[0] > buff:
            ecm.dmove_joint_one(delta, 0)
        elif pos_camera[0] < - buff:
            ecm.dmove_joint_one(-delta, 0)
        else:
            xcentered = True
        
        if pos_camera[1] > buff:
            ecm.dmove_joint_one(delta, 1)
        elif pos_camera[1] < - buff:
            ecm.dmove_joint_one(-delta, 1)
        else:
            ycentered = True
            
def two_bowls_two_psms(bowl_p1, ball_p1, bowl_p2, ball_p2):
    global psm1, psm2
    psm1_distance_bowl1, psm1_distance_bowl2 = distance(bowl_p1)
    psm2_distance_bowl1, psm2_distance_bowl2 = distance(bowl_p2)

    psm1_distance_ball1, psm1_distance_ball2 = distance(ball_p1)
    psm2_distance_ball1, psm2_distance_ball2 = distance(ball_p2)
   
    if psm1_distance_bowl1 < psm2_distance_bowl1:
        psm1_bowl = bowl_p1[0]
        psm2_bowl = bowl_p2[1]
    else:
        psm1_bowl = bowl_p1[1]
        psm2_bowl = bowl_p2[0]

    if psm1_distance_ball1 < psm2_distance_ball1:
        psm1_ball = ball_p1[0]
        psm2_ball = ball_p2[1]
    else:
        psm1_ball = ball_p1[1]
        psm2_ball = ball_p2[0]
    try:
        thread.start_new_thread(pick_and_place,(psm1, psm1_ball, psm1_bowl))
        thread.start_new_thread(pick_and_place,(psm2, psm2_ball, psm2_bowl))
    except:
        print("Error launching threads")
    while 1:
        pass
    
def pick_and_place_one(balls, bowls, psm): #one bowl one psm multiple balls
    i = 0
    for ball in balls:
        x = threading.Thread(target=pick_and_place, args=(psm, balls[i], bowls))
        x.start()
        while x.is_alive():
            pass
        i+=1

def pick_and_place_multiple(balls1, balls2, bowl1, bowl2, psm1, psm2):
        x = threading.Thread(target=pick_and_place_one, args=(balls1, bowl1, psm1))
        y = threading.Thread(target=pick_and_place_one, args=(balls2, bowl2, psm2))
        x.start()
        y.start()
        
def switch(larger_set, smaller_set, difference, even):
    i = 0
    
    if smaller_set == psm1_balls:
        larger_set.sort(key=lambda x: x.bowl1_distance, reverse = False)
        print('distance form bowl1')
        for bal in larger_set:
            print(bal.bowl1_distance)
            print(bal)
        print('___________________')
    elif smaller_set == psm2_balls:
        larger_set.sort(key=lambda x: x.bowl2_distance)
    
    if even:
        delta = abs(difference) - 1
    elif not even:
        delta = abs(difference) - 2
        
    
    moved_set = larger_set[0:delta]
    larger_set = larger_set[delta:len(larger_set)]
    #moved_set = larger_set[0:1]
    #larger_set = larger_set[1:2]
    for element in moved_set:
        smaller_set.append(element)
        
    return larger_set, smaller_set
        
        

In [4]:
def pick_and_place_two(bowls, balls): #two bowls two psms multiple balls
    #find distance between bowls and one psm and assign one bowl to one psm
    global psm1_balls, psm2_balls
    
    bowls[0].psm1_distance = distance(bowls[0].psm1_coordinates,[0,0,0])
    bowls[0].psm2_distance = distance(bowls[0].psm2_coordinates,[0,0,0])
    bowls[1].psm2_distance = distance(bowls[1].psm1_coordinates,[0,0,0])
    bowls[1].psm2_distance = distance(bowls[1].psm2_coordinates,[0,0,0])
   

    if bowls[0].psm1_distance < bowls[0].psm2_distance:
        psm1_bowl = bowls[0]
        psm2_bowl = bowls[1]
    else:
        psm1_bowl = bowls[1]
        psm2_bowl = bowls[0]
                                                            
    print('1- psm1_bowl')
    print(psm1_bowl)
    print('psm2_bowl')
    print(psm2_bowl)
    
    psm1_balls = []
    psm2_balls = []
                                                        
    for ball in balls:
    
        ball.bowl1_distance = distance(ball.world_coordinates, psm1_bowl.world_coordinates)
        ball.bowl2_distance = distance(ball.world_coordinates, psm2_bowl.world_coordinates)
    
        if ball.bowl1_distance < ball.bowl2_distance:
            psm1_balls.append(ball)
        else:
            psm2_balls.append(ball)
            
    print('2- psm1_ball')
    print(psm1_balls)
    print('psm2_ball')
    print(psm2_balls)
    
    difference = len(psm1_balls) - len(psm2_balls)

    if len(balls) % 2 == 0:
        even = True
    else:
        even = False

    if difference > 1:
        psm1_balls, psm2_balls = switch(psm1_balls, psm2_balls, difference, even)
    elif difference < -1:
        psm2_balls, psm1_balls = switch(psm2_balls, psm1_balls, difference, even)
        
    print('3- psm1_ball')
    print(psm1_balls)
    print('psm2_ball')
    print(psm2_balls)
    
    pick_and_place_multiple(psm1_balls, psm2_balls, psm1_bowl, psm2_bowl, psm1, psm2)

In [5]:
tf_listener = tf.TransformListener()
time.sleep(10)
tf_listener.getFrameStrings()
#https://answers.ros.org/question/370813/waiting-for-transformlistener-with-ros2-and-python/

['J1_ECM',
 'PSM2_tool_wrist_shaft_link',
 'PSM2_outer_pitch_front_link',
 'PSM1_RCM',
 'PSM2_tool_main_link',
 'PSM2_tool_wrist_caudier_link_2_left',
 'PSM1',
 'PSM2',
 'PSM3',
 'PSM1_tool_wrist_sca_ee_link_0',
 'PSM1_tool_wrist_sca_ee_link_1',
 'PSM1_tool_wrist_sca_ee_link_2',
 'ecm_setup_link',
 'ECM_RCM',
 'ecm_yaw_link',
 'PSM2_tool_wrist_caudier_link_shaft',
 'PSM2_tool_wrist_caudier_ee_link',
 'ECM',
 'ecm_pitch_link_1',
 'PSM1_outer_pitch_link',
 'PSM2_base',
 'PSM2_outer_pitch_link',
 'PSM1_tool_tip_link',
 'camera',
 'PSM1_outer_pitch_front_link',
 'ecm_pitch_link',
 'PSM1_outer_pitch_top_link',
 'PSM2_RCM',
 'PSM2_tool_wrist_caudier_link',
 'PSM1_outer_yaw_link',
 'Jp21_ECM',
 'ecm_pitch_link_2',
 'ecm_pitch_link_3',
 'PSM1_tool_wrist_shaft_link',
 'PSM2_outer_pitch_back_link',
 'PSM1_tool_wrist_sca_link',
 'PSM2_outer_pitch_top_link',
 'PSM3_base',
 'PSM2_tool_wrist_caudier_ee_link_0',
 'PSM1_outer_insertion_link',
 'PSM2_outer_pitch_bottom_link',
 'ECM_base',
 'ecm_inserti

In [23]:
rospy.Subscriber('/stereo/left/image_raw', msg.Image, left_image_callback)
rospy.Subscriber('/stereo/left/camera_info', msg.CameraInfo, left_camera_info_callback)
rospy.Subscriber('/stereo/right/image_raw', msg.Image, right_image_callback)
rospy.Subscriber('/stereo/right/camera_info', msg.CameraInfo, right_camera_info_callback)

#images = np.concatenate((left_image,right_image),axis = 1)
#cv2.imshow("images",images)
#cv2.waitKey(0)
#plt.imshow(images)

dimensions = right_image.shape
height = right_image.shape[0]
width = right_image.shape[1]
    
update_transforms()

stereo_cam = image_geometry.StereoCameraModel()
stereo_cam.fromCameraInfo(left_camera_info, right_camera_info)

In [24]:
print(left_camera_info)
print('________________')
print(left_camera_info)

header: 
  seq: 2065
  stamp: 
    secs: 1632774265
    nsecs: 633857727
  frame_id: "left_camera"
height: 288
width: 360
distortion_model: "plumb_bob"
D: [0.0, 0.0, 0.0, 0.0, 0.0]
K: [311.9603517065951, 0.0, 180.0, 0.0, 311.9603517065951, 144.0, 0.0, 0.0, 1.0]
R: [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
P: [311.9603517065951, 0.0, 180.0, 0.0, 0.0, 311.9603517065951, 144.0, 0.0, 0.0, 0.0, 1.0, 0.0]
binning_x: 1
binning_y: 1
roi: 
  x_offset: 0
  y_offset: 0
  height: 0
  width: 0
  do_rectify: True
________________
header: 
  seq: 2065
  stamp: 
    secs: 1632774265
    nsecs: 633857727
  frame_id: "left_camera"
height: 288
width: 360
distortion_model: "plumb_bob"
D: [0.0, 0.0, 0.0, 0.0, 0.0]
K: [311.9603517065951, 0.0, 180.0, 0.0, 311.9603517065951, 144.0, 0.0, 0.0, 1.0]
R: [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
P: [311.9603517065951, 0.0, 180.0, 0.0, 0.0, 311.9603517065951, 144.0, 0.0, 0.0, 0.0, 1.0, 0.0]
binning_x: 1
binning_y: 1
roi: 
  x_offset: 0
  y_offset: 0
  heigh

In [25]:
update_transforms()
#balls, bowls = images_to_object_positions(left_image, right_image)
#center_frame(balls[0].world_coordinates, 0.005, 0.005)
#world_position_to_joints(balls)
#world_position_to_joints(bowls)
#pick_and_place_one(balls, bowls[0], psm2)
#pick_and_place_two(bowls, balls)


4981.0
6865.0
631.5
2.0
671.0
759.0
2.0
777.5
balls
[]
bowls
[[179, 264], [303, 254]]
4667.5
6738.0
636.5
661.0
766.5
772.5
balls
[]
bowls
[[169, 265], [295, 256]]
[    -1.45124,  0.00489409,    0.691712]
[    -1.42667,   -0.073543,    0.662935]
1- psm1_bowl
<__main__.Item instance at 0x7fa4ed09deb0>
psm2_bowl
<__main__.Item instance at 0x7fa4ecff2dc0>
2- psm1_ball
[]
psm2_ball
[]
3- psm1_ball
[]
psm2_ball
[]


[ERROR] [1632774267.709562]: bad callback: <bound method ecm.__state_joint_current_cb of <dvrk.ecm.ecm object at 0x7fa4ec797550>>
Traceback (most recent call last):
  File "/opt/ros/melodic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/home/kareem/dvrk_ws/src/dvrk-ros/dvrk_python/src/dvrk/arm.py", line 286, in __state_joint_current_cb
    self.__effort_joint_current.resize(len(data.effort))
ValueError: cannot resize an array that references or is referenced
by another array in this way.  Use the resize function

[ERROR] [1632774278.473622]: bad callback: <bound method psm.__state_joint_current_cb of <dvrk.psm.psm object at 0x7fa4ec145b50>>
Traceback (most recent call last):
  File "/opt/ros/melodic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/home/kareem/dvrk_ws/src/dvrk-ros/dvrk_python/src/dvrk/arm.py", line 286, in __state_joint_current_cb
    self.__effort_joint_current.resize(len(